In [1]:
from rgb_anything_v1.dpt import RGBAnything
from torch import nn
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import Compose
from cocoloader import cocoloader

xFormers not available
xFormers not available


In [4]:
model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}


net = RGBAnything(**model_configs['vits'])
net._init_weights()

In [5]:
train_iter = cocoloader('data/train_small', batch_size=4)
batch = next(iter(train_iter))

In [6]:
# plt.imshow(np.array(batch[0][0][0]), cmap='Reds')

In [7]:
x = batch[0].to(torch.device('cuda'))
net = net.to(torch.device('cuda'))

In [6]:
with torch.no_grad():
    y = net(x)

In [7]:
y_true = batch[1].to(torch.device('cuda'))

In [11]:
loss = nn.MSELoss()
loss(y, y_true)

tensor(1.4828, device='cuda:0')

In [ ]:
x.shape, y.shape, y_true.shape

(torch.Size([4, 2, 518, 518]),
 torch.Size([4, 518, 518]),
 torch.Size([4, 518, 518]))

In [ ]:
def absrel(y_hat, y, threshold = 5):
    denorm = lambda x: ((x * 0.224 + 0.456) * 255).clamp(0, 255).to(torch.uint8).to(torch.float32)  
    y, y_hat = denorm(y), denorm(y_hat)
    mask = y >= threshold
    abs_rel = torch.mean(torch.abs(y_hat[mask] - y[mask]) / y[mask])
    return abs_rel.item()

In [44]:
a = torch.tensor([10, 20, 30])
b = torch.tensor([0, 30, 45])
absrel(norm(a), norm(b))  # should be 0.0333

0.350191593170166

In [37]:
norm = lambda x: (x / 255.0 - 0.456) / 0.224
denorm = lambda x: ((x * 0.224 + 0.456) * 255).clamp(0, 255)
mask = denorm(norm(b)) >= 5

In [39]:
(a[mask]-b[mask]) / b[mask]  # should be [0.0333]

tensor([-0.3333, -0.3333])